## Generate Hyp3 interferogram stack

This notebook will queue up processing jobs in Hyp3 and then download and prepare them for use in Mintpy. Code adapted from [Hyp3 tutorial](https://nbviewer.org/github/ASFHyP3/hyp3-docs/blob/main/docs/tutorials/hyp3_insar_stack_for_ts_analysis.ipynb).


In [10]:
site = 'NorthSlopeEastD102'
year = 2024

### Prepare environment

Import functions and load metadata for processing. The environment requires several packages from the Alaska Satellite Facility to function, specifically [asf_search](https://docs.asf.alaska.edu/asf_search/basics/) and [hyp3_sdk](https://hyp3-docs.asf.alaska.edu/using/sdk/).

In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
from dateutil.parser import parse as parse_date
from typing import List, Union
from osgeo import gdal
from solid_utils import permafrost_utils as pu
import asf_search as asf
import hyp3_sdk as sdk

In [12]:
year

2024

In [ ]:
################# Set Directories ##########################################
print('\nCurrent directory:',os.getcwd())

if 'work_dir' not in locals():
    work_dir = Path.cwd()/'work'/'permafrost_ouputs'/site/str(year)

print("Work directory:", work_dir)
work_dir.mkdir(parents=True, exist_ok=True)
# Change to Workdir   
os.chdir(work_dir)
       
hyp3_dir = work_dir/'products'               #aka gunwdir
hyp3_dir.mkdir(parents=True, exist_ok=True)
print("   Hyp3  dir:", hyp3_dir) 
    
mintpy_dir = work_dir/'MintPy' 
mintpy_dir.mkdir(parents=True, exist_ok=True)
print("   MintPy  dir:", mintpy_dir)
############################################################################
### List of CalVal Sites:
'''
Set NISAR calval sites:
    NorthSlopeEastD102      : North Slope of Alaska including Dalton highway, Sentinel-1 descending path (track) 102. 
                            : The field validation sites are located in this frame.
    NorthSlopeWestD44       : Western area of the North Slope of Alaska, Sentinel-1 descending path (track) 44.
    NorthwestTerritoriesA78 : North east of Yellowknife, Sentinel-1 ascending path (track) 78.


Hyp3 & MintPy parameters:
    calval_location    : name
    region_identifier  : WTK string with latlon point for identifying image in Hyp3
    subset_region      : subset analysis area, in UTM. Given in '[ymin:ymax,xmin:xmax]' or 'none'
    download_start_date : download start date as YYYMMDD  
    download_end_date   : download end date as YYYMMDD
    mintpy_ref_loc     : reference point for use in mintpy in UTM Y,X order. Projection must be that of the Hyp3 images.
    tempBaseMax        : maximum number of days, don't use interferograms longer than this value 
    ifgExcludeList     : default is not to exclude any interferograms
    maskWater          : interior locations don't need to mask water
    sentinel_path      : asfPath number for identifying image. Also called track.
    sentinel_frame     : asfFrame number for identifying image
'''
sites = {
    ##########  NORTH SLOPE EAST (DALTON/TOOLIK) ##############
    'NorthSlopeEastD102' : {'calval_location' : 'NorthSlopeEastD102',
            'region_identifier' : 'POINT(-149.37 69.09)',
            'subset_region' : '[7620213:7686754, 641941:679925]',            
            'download_start_date' : '20230525',
            'download_end_date'   : '20230910',
            'mintpy_ref_loc'      : '7651392, 666923',
            'tempBaseMax' : '36',
            'ifgExcludeList' : 'auto',
            'maskWater' : 'True',
            'sentinel_direction' : 'DESCENDING',
            'sentinel_path' : '102',
            'sentinel_frame' : '362'},
    'NorthSlopeWestD44' : {'calval_location' : 'NorthSlopeWestD44',
            'region_identifier' : 'POINT(-160.25 68.82)',
            'subset_region' : 'none',
            'download_start_date' : '20230525',
            'download_end_date'   : '20230910',
            'mintpy_ref_loc' : '7684081, 437118',
            'tempBaseMax' : '36',
            'ifgExcludeList' : 'auto',
            'maskWater' : 'True',
            'sentinel_direction' : 'DESCENDING',
            'sentinel_path' : '44',
            'sentinel_frame' : '362'},
    'NorthwestTerritoriesA78' : {'calval_location' : 'NorthwestTerritoriesA78',
            'region_identifier' : 'POINT(-111.35 63.20)',
            'subset_region' : 'none',
            'download_start_date' : '20230525',
            'download_end_date'   : '20230910',
            'mintpy_ref_loc'      : '6980804, 522414',
            'tempBaseMax' : '36',
            'ifgExcludeList' : 'auto',
            'maskWater' : 'True',
            'sentinel_direction' : 'ASCENDING',
            'sentinel_path' : '78',
            'sentinel_frame' : '204'},    
}


Current directory: /home/jovyan/forks/ATBD/methods/permafrost/work/permafrost_ouputs/NorthSlopeEastD102/2023
Work directory: /home/jovyan/forks/ATBD/methods/permafrost/work/permafrost_ouputs/NorthSlopeEastD102/2023
   Hyp3  dir: /home/jovyan/forks/ATBD/methods/permafrost/work/permafrost_ouputs/NorthSlopeEastD102/2023/products
   MintPy  dir: /home/jovyan/forks/ATBD/methods/permafrost/work/permafrost_ouputs/NorthSlopeEastD102/2023/MintPy


### Generate SBAS image pairs

Identify stack of SLC images and determine SBAS pairs.

In [21]:
lonlat = sites[site]['region_identifier']
frame = sites[site]['sentinel_frame']
if sites[site]['sentinel_direction']=='ASCENDING':
    direction = asf.constants.ASCENDING
elif sites[site]['sentinel_direction']=='DESCENDING':
    direction = asf.constants.DESCENDING

stack_start = parse_date(f'{year}-05-25 00:00:00Z')
stack_end = parse_date(f'{year}-09-10 00:00:00Z')

search_results = asf.geo_search(platform=asf.constants.SENTINEL1, 
                                intersectsWith=lonlat, 
                                start=stack_start,
                                end=stack_end,
                                processingLevel=asf.constants.SLC,
                                beamMode=asf.constants.IW,
                                flightDirection=direction,
                                asfFrame=int(frame))

In [22]:
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

# stack_start = parse_date(f'{year}-05-25 00:00:00Z')
# stack_end = parse_date(f'{year}-09-10 00:00:00Z')

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

stack

,centerLat,centerLon,stopTime,fileID,flightDirection,pathNumber,processingLevel,url,startTime,sceneName,...,processingDate,sensor,groupID,pgeVersion,fileName,beamModeType,s3Urls,temporalBaseline,perpendicularBaseline,geometry
210,68.9319,-151.7626,2024-05-28T16:44:32Z,S1A_IW_SLC__1SDV_20240528T164404_20240528T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2024-05-28 16:44:04+00:00,S1A_IW_SLC__1SDV_20240528T164404_20240528T1644...,...,2024-05-28T16:44:04Z,C-SAR,S1A_IWDV_0361_0368_054074_102,003.71,S1A_IW_SLC__1SDV_20240528T164404_20240528T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,0,0.0,"{'coordinates': [[[-149.299286, 67.836052], [-..."
211,68.9320,-151.7643,2024-06-09T16:44:32Z,S1A_IW_SLC__1SDV_20240609T164404_20240609T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2024-06-09 16:44:04+00:00,S1A_IW_SLC__1SDV_20240609T164404_20240609T1644...,...,2024-06-09T16:44:04Z,C-SAR,S1A_IWDV_0361_0368_054249_102,003.71,S1A_IW_SLC__1SDV_20240609T164404_20240609T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,12,69.0,"{'coordinates': [[[-149.300827, 67.836243], [-..."
212,68.5232,-152.0280,2024-06-21T16:44:38Z,S1A_IW_SLC__1SDV_20240621T164411_20240621T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2024-06-21 16:44:11+00:00,S1A_IW_SLC__1SDV_20240621T164411_20240621T1644...,...,2024-06-21T16:44:11Z,C-SAR,S1A_IWDV_0363_0369_054424_102,003.80,S1A_IW_SLC__1SDV_20240621T164411_20240621T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,24,143.0,"{'coordinates': [[[-149.582291, 67.457756], [-..."
213,68.9326,-151.7634,2024-07-03T16:44:30Z,S1A_IW_SLC__1SDV_20240703T164402_20240703T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2024-07-03 16:44:02+00:00,S1A_IW_SLC__1SDV_20240703T164402_20240703T1644...,...,2024-07-03T16:44:02Z,C-SAR,S1A_IWDV_0361_0368_054599_102,003.80,S1A_IW_SLC__1SDV_20240703T164402_20240703T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,36,78.0,"{'coordinates': [[[-149.299759, 67.836845], [-..."
214,68.9320,-151.7579,2024-07-27T16:44:29Z,S1A_IW_SLC__1SDV_20240727T164402_20240727T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2024-07-27 16:44:02+00:00,S1A_IW_SLC__1SDV_20240727T164402_20240727T1644...,...,2024-07-27T16:44:02Z,C-SAR,S1A_IWDV_0361_0368_054949_102,003.80,S1A_IW_SLC__1SDV_20240727T164402_20240727T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,60,-149.0,"{'coordinates': [[[-149.294571, 67.836044], [-..."
215,68.9319,-151.7596,2024-08-08T16:44:30Z,S1A_IW_SLC__1SDV_20240808T164402_20240808T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2024-08-08 16:44:02+00:00,S1A_IW_SLC__1SDV_20240808T164402_20240808T1644...,...,2024-08-08T16:44:02Z,C-SAR,S1A_IWDV_0361_0368_055124_102,003.80,S1A_IW_SLC__1SDV_20240808T164402_20240808T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,72,-113.0,"{'coordinates': [[[-149.296417, 67.836014], [-..."
216,68.9315,-151.7604,2024-08-20T16:44:30Z,S1A_IW_SLC__1SDV_20240820T164402_20240820T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2024-08-20 16:44:02+00:00,S1A_IW_SLC__1SDV_20240820T164402_20240820T1644...,...,2024-08-20T16:44:02Z,C-SAR,S1A_IWDV_0361_0368_055299_102,003.80,S1A_IW_SLC__1SDV_20240820T164402_20240820T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,84,-118.0,"{'coordinates': [[[-149.297394, 67.835625], [-..."
217,68.9315,-151.7566,2024-09-01T16:44:30Z,S1A_IW_SLC__1SDV_20240901T164402_20240901T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2024-09-01 16:44:02+00:00,S1A_IW_SLC__1SDV_20240901T164402_20240901T1644...,...,2024-09-01T16:44:02Z,C-SAR,S1A_IWDV_0361_0368_055474_102,003.80,S1A_IW_SLC__1SDV_20240901T164402_20240901T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,96,-212.0,"{'coordinates': [[[-149.293549, 67.83564], [-1..."


In [23]:
sbas_pairs = set()

for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack.loc[
        (stack.sceneName != reference)
        & (stack.temporalBaseline - rt <= int(sites[site]['tempBaseMax']))
        & (stack.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))

### Do Hyp3 Processing

This step requires NASA Earthdata credentials. See [Hyp3 documentation](https://hyp3-docs.asf.alaska.edu/) for more information about using Hyp3 processing. Each user has an allotted Hyp3 processing quota, for which some of this processing is counted against.

In [24]:

hyp3 = sdk.HyP3(prompt=True)


NASA Earthdata Login username:  acjohnson16
NASA Earthdata Login password:  ········


In [25]:

jobs = sdk.Batch()
h3projname = f"{site}_NCV_{year}"
for reference, secondary in sbas_pairs:
    jobs += hyp3.submit_insar_job(reference, secondary, name=h3projname,
                                  include_dem=True, include_look_vectors=True)


This processing will likely take about an hour to complete. You cannot download data until the processing is finished. The following cell will check to see if the processing is still running.

In [30]:
# class stillrunningError(Error):
#     """raised when jobs are still running"""
#     pass

running = len(hyp3.find_jobs(status_code=["RUNNING"]))
pending = len(hyp3.find_jobs(status_code=["PENDING"]))

# if running > 0:
#     print('Jobs still running!')
#     # raise stillrunningError
if running+pending ==0:
    print('No jobs running, all clear')
else:
    print(f'Jobs still running!\n{pending} pending, {running} running')

No jobs running, all clear


### Download Data

In [31]:

h3projname = f"{site}_NCV_{year}"
batch = hyp3.find_jobs(name=h3projname)
insar_products = batch.download_files(hyp3_dir)
insar_products = [sdk.util.extract_zipped_product(ii) for ii in insar_products]


  0%|          | 0/15 [00:00<?, ?it/s]
S1AA_20240727T164402_20240808T164402_VVP012_INT80_G_ueF_73F0.zip:   0%|          | 0/222089626 [00:00<?, ?it/s]
S1AA_20240727T164402_20240808T164402_VVP012_INT80_G_ueF_73F0.zip:   9%|▉         | 20.0M/212M [00:00<00:04, 46.7MB/s]
S1AA_20240727T164402_20240808T164402_VVP012_INT80_G_ueF_73F0.zip:  14%|█▍        | 30.0M/212M [00:00<00:04, 43.5MB/s]
S1AA_20240727T164402_20240808T164402_VVP012_INT80_G_ueF_73F0.zip:  19%|█▉        | 40.0M/212M [00:01<00:04, 38.1MB/s]
S1AA_20240727T164402_20240808T164402_VVP012_INT80_G_ueF_73F0.zip:  24%|██▎       | 50.0M/212M [00:01<00:04, 41.9MB/s]
S1AA_20240727T164402_20240808T164402_VVP012_INT80_G_ueF_73F0.zip:  28%|██▊       | 60.0M/212M [00:01<00:04, 35.1MB/s]
S1AA_20240727T164402_20240808T164402_VVP012_INT80_G_ueF_73F0.zip:  33%|███▎      | 70.0M/212M [00:01<00:03, 39.6MB/s]
S1AA_20240727T164402_20240808T164402_VVP012_INT80_G_ueF_73F0.zip:  38%|███▊      | 80.0M/212M [00:02<00:03, 38.8MB/s]
S1AA_20240727T164402_20

### Crop Interferograms

In [32]:
# Crop to common overlap
files = hyp3_dir.glob('*/*_dem.tif')
overlap = pu.get_common_overlap(files)
pu.clip_hyp3_products_to_common_overlap(hyp3_dir, overlap)

/home/jovyan/.local/envs/solid_earth_atbd/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
